
# 🎬 Movie Revenue Prediction — Colab Notebook

This notebook prepares a **clean, slim dataset** from TMDB 5000 Movies & Credits, then trains two models:

- **Baseline:** Linear Regression
- **Model 2:** Random Forest Regressor

We evaluate with:
- **RMSE** (Root Mean Squared Error)
- **MAE** (Mean Absolute Error)
- **R² Score**

It also saves lightweight artifacts (dataset + models) **under 25MB** for GitHub/Streamlit deployment.


In [1]:

# If you're in Colab, uncomment the next line to ensure packages are available.
# !pip -q install scikit-learn pandas numpy joblib

import os, json, ast, math, joblib
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Paths (change if needed). In Colab, upload the CSVs if files not found.
MOVIES_PATH = 'tmdb_5000_movies.csv'
CREDITS_PATH = 'tmdb_5000_credits.csv'

if not (os.path.exists(MOVIES_PATH) and os.path.exists(CREDITS_PATH)):
    try:
        # Colab-friendly upload
        from google.colab import files
        print("Please upload tmdb_5000_movies.csv and tmdb_5000_credits.csv")
        uploaded = files.upload()
        # Files will be saved to the current directory
    except Exception as e:
        print("Upload helper unavailable. Ensure the files are present in the working directory.")
        raise e


Please upload tmdb_5000_movies.csv and tmdb_5000_credits.csv


Saving tmdb_5000_credits.csv to tmdb_5000_credits.csv
Saving tmdb_5000_movies.csv to tmdb_5000_movies.csv



## Preprocessing & Feature Engineering

- Merge movies and credits on **title**
- Parse JSON-like columns (counts only, for compactness)
- Extract **release_year**
- Keep numeric predictors and engineered counts
- Drop rows with **missing revenue** or clearly invalid values
- Compute feature correlations with revenue and keep **top K** features


In [2]:

def safe_json_count(x):
    # TMDB stores JSON-like strings; count elements if parsable
    try:
        arr = json.loads(x) if isinstance(x, str) else (x or [])
        return len(arr) if isinstance(arr, list) else 0
    except Exception:
        return 0

def safe_parse_date(d):
    try:
        return int(pd.to_datetime(d, errors='coerce').year)
    except Exception:
        return np.nan

def load_and_merge(movies_path, credits_path):
    movies = pd.read_csv(movies_path)
    credits = pd.read_csv(credits_path)
    # Ensure consistent key
    movies['title'] = movies['title'].astype(str)
    credits['title'] = credits['title'].astype(str)
    df = movies.merge(credits, on='title', how='inner', suffixes=('', '_credits'))
    return df

def engineer_features(df):
    # JSON-count features
    json_cols = ['genres', 'keywords', 'production_companies', 'production_countries', 'spoken_languages']
    for c in json_cols:
        if c in df.columns:
            df[c + '_count'] = df[c].apply(safe_json_count)

    # Credits: cast/crew counts & director writers
    for c in ['cast', 'crew']:
        if c in df.columns:
            df[c + '_count'] = df[c].apply(safe_json_count)

    # simple director flag: presence of a Director in crew json (count of directors)
    def count_directors(crew_str):
        try:
            items = json.loads(crew_str) if isinstance(crew_str, str) else []
            return sum(1 for it in items if isinstance(it, dict) and it.get('job')=='Director')
        except Exception:
            return 0
    if 'crew' in df.columns:
        df['director_count'] = df['crew'].apply(count_directors)
    else:
        df['director_count'] = 0

    # release year
    if 'release_date' in df.columns:
        df['release_year'] = pd.to_datetime(df['release_date'], errors='coerce').dt.year
    else:
        df['release_year'] = np.nan

    # Select candidate numeric features
    candidate_numeric = [
        'budget','popularity','runtime','vote_average','vote_count',
        'genres_count','keywords_count','production_companies_count',
        'production_countries_count','spoken_languages_count',
        'cast_count','crew_count','director_count','release_year'
    ]
    available_features = [c for c in candidate_numeric if c in df.columns]

    # Target
    target = 'revenue'
    # Clean
    out = df[available_features + [target]].copy()
    # Handle nonsensical values
    out.replace([np.inf, -np.inf], np.nan, inplace=True)
    out.dropna(subset=[target], inplace=True)
    # Remove rows with revenue <= 0 (likely unknown)
    out = out[out[target] > 0]
    # Fill missing numerics with median
    for c in available_features:
        if out[c].isna().any():
            out[c] = out[c].fillna(out[c].median())

    return out, available_features, target

def top_k_by_corr(df, target, features, k=12):
    corr = df[features + [target]].corr(numeric_only=True)[target].drop(target)
    corr_abs = corr.abs().sort_values(ascending=False)
    top_feats = list(corr_abs.head(k).index)
    return top_feats, corr, corr_abs

df_all = load_and_merge(MOVIES_PATH, CREDITS_PATH)
df_proc, feature_pool, TARGET = engineer_features(df_all)

TOP_K = 12  # adjustable
top_features, corr, corr_abs = top_k_by_corr(df_proc, TARGET, feature_pool, k=TOP_K)

print("Top features by |correlation| with revenue:")
display(pd.DataFrame({
    'feature': corr_abs.index,
    'abs_corr_with_revenue': corr_abs.values
}).reset_index(drop=True).head(TOP_K))

# Identify single highest-correlation feature (unique strongest signal)
unique_top_feature = corr_abs.index[0] if len(corr_abs)>0 else None
print("\nHighest-correlation single feature:", unique_top_feature)


Top features by |correlation| with revenue:


,feature,abs_corr_with_revenue
0,vote_count,0.758257
1,budget,0.707866
2,popularity,0.605897
3,crew_count,0.363863
4,cast_count,0.336079
5,runtime,0.233670
6,keywords_count,0.199783
7,vote_average,0.180789
8,genres_count,0.170776
9,release_year,0.137052



Highest-correlation single feature: vote_count



## Train/Test Split & Models

We use an 80/20 split, with a standardization step for Linear Regression.
Random Forest uses modest depth/trees to keep artifacts small while performing well.


In [5]:

SEED = 42

X = df_proc[top_features].copy()
y = df_proc[TARGET].copy()

# Optionally log-transform target for stability (commented out to keep target in original scale)
# y = np.log1p(y)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=SEED
)

# Pipelines
lin_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', LinearRegression())
])

rf_pipeline = Pipeline([
    ('model', RandomForestRegressor(
        n_estimators=120,
        max_depth=12,
        min_samples_split=4,
        min_samples_leaf=2,
        random_state=SEED,
        n_jobs=-1
    ))
])

lin_pipeline.fit(X_train, y_train)
rf_pipeline.fit(X_train, y_train)

def evaluate(model, name):
    preds = model.predict(X_test)
    mse = mean_squared_error(y_test, preds)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, preds)
    r2 = r2_score(y_test, preds)
    return {
        'model': name,
        'RMSE': rmse,
        'MAE': mae,
        'R2': r2
    }

results = pd.DataFrame([
    evaluate(lin_pipeline, 'Linear Regression'),
    evaluate(rf_pipeline, 'Random Forest')
])

print("Success Metrics on Test Set:")
display(results)


Success Metrics on Test Set:


,model,RMSE,MAE,R2
0,Linear Regression,1.271461e+08,6.423604e+07,0.682112
1,Random Forest,1.193950e+08,5.861365e+07,0.719689



## Save Compact Artifacts (GitHub/Streamlit-friendly)

Files saved:
- `dataset_preprocessed_topk.csv` — **slim dataset** with selected features + target
- `feature_list.json` — features used (in order of absolute correlation)
- `linreg_pipeline.joblib` — linear regression pipeline (scaler + model)
- `rf_pipeline.joblib` — random forest model

> All kept compact to comfortably stay under 25MB.


In [6]:

ART_DIR = "artifacts"
os.makedirs(ART_DIR, exist_ok=True)

# Save slim dataset
slim_df = df_proc[top_features + [TARGET]].copy()
slim_path = os.path.join(ART_DIR, "dataset_preprocessed_topk.csv")
slim_df.to_csv(slim_path, index=False)

# Save feature list
with open(os.path.join(ART_DIR, "feature_list.json"), "w") as f:
    json.dump({'features': top_features, 'target': TARGET, 'highest_corr_feature': top_features[0] if len(top_features)>0 else None}, f, indent=2)

# Save models
joblib.dump(lin_pipeline, os.path.join(ART_DIR, "linreg_pipeline.joblib"))
joblib.dump(rf_pipeline, os.path.join(ART_DIR, "rf_pipeline.joblib"))

def sizeof_mb(path):
    return os.path.getsize(path) / (1024*1024)

sizes = {fn: round(sizeof_mb(os.path.join(ART_DIR, fn)), 3) for fn in os.listdir(ART_DIR)}
print("Saved files and sizes (MB):")
print(sizes)
total_size = sum(sizes.values())
print(f"Total artifacts size: {round(total_size, 3)} MB")


Saved files and sizes (MB):
{'rf_pipeline.joblib': 7.544, 'linreg_pipeline.joblib': 0.002, 'dataset_preprocessed_topk.csv': 0.201, 'feature_list.json': 0.0}
Total artifacts size: 7.747 MB



## Streamlit Snippet (minimal)

```python
# streamlit_app.py
import streamlit as st
import pandas as pd
import joblib, json

st.title("🎬 Movie Revenue Prediction")

# Load artifacts
features = json.load(open("artifacts/feature_list.json"))["features"]
model = joblib.load("artifacts/rf_pipeline.joblib")

# Inputs
vals = {}
for f in features:
    vals[f] = st.number_input(f, value=0.0)
X = pd.DataFrame([vals])

# Predict
pred = model.predict(X)[0]
st.metric("Predicted Revenue", f"${pred:,.0f}")
```

Run with:
```bash
streamlit run streamlit_app.py
```
